Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import math
from copy import copy
from functools import reduce
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse

In [ ]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [ ]:
x = make_set_covering_problem(100, 100, .3)
print("Element at row=42 and column=42:", x[42, 42])

In [ ]:
PROBLEM_SIZE = 1000
NUM_SETS = 1000
SETS_TRUE_PROBABILITY = 0.3

In [ ]:
SETS = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, SETS_TRUE_PROBABILITY)

In [ ]:
SETS.A

In [ ]:
def fitness(state):
    result_coverage = coverage(state)
    return -(state.sum()+ (result_coverage == False).sum() *  PROBLEM_SIZE)   

def coverage(state):
    return (SETS * state).sum(axis=0)

In [ ]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [ ]:
def tweak_and_feat(state, state_fitness):
    index = randint(0, NUM_SETS - 1)
    state[index] = not state[index]
    new_fitness = fitness(state)
    if new_fitness > state_fitness:
        return True, state, new_fitness
    else:
        state[index] = not state[index]
        return False, state, state_fitness

# Hill climber

In [ ]:
# random() < 0.005
current_state = [False for _ in range(NUM_SETS)]
current_state = np.expand_dims(current_state, 0).transpose()
fitness_current = fitness(current_state)
print(f"Start fitness {fitness_current}")

max_step = 5_000
count_steady = max_step/10
count_equal = 0

In [ ]:
for step in range(max_step):
    count_equal += 1
    res, st, fit = tweak_and_feat(current_state, fitness_current)
    if res:
        count_equal = 0
        current_state, fitness_current = st, fit
        print(f"Fit {fitness_current} Step {step}")
    if count_equal > count_steady:
        print("steady state reached")
        break

print(f"Resolved {(coverage(current_state) == False).sum() == 0} with {current_state.sum()} in {step} step")
coverage(current_state)

# Hill climber best of three

In [ ]:
current_state = [False for _ in range(NUM_SETS)]
current_state = np.expand_dims(current_state, 0).transpose()
fitness_current = fitness(current_state)
print(f"Start fitness {fitness_current}")

max_step = 5_000
count_steady = max_step/10
count_equal = 0

In [ ]:
for step in range(max_step):
    count_equal += 1
    state_new = tweak(current_state)
    fitness_new = fitness(state_new)
    for _ in range(4):
        s_new = tweak(current_state)
        f_new = fitness(state_new)
        if f_new > fitness_new:
            state_new, fitness_new = s_new, f_new
    
    if fitness_new > fitness_current:
        count_equal = 0
        current_state, fitness_current = state_new, fitness_new
        print(f"Fit {fitness_current} Step {step}")
    if count_equal > count_steady:
        print("steady state reached")
        break

print(f"Resolved {(coverage(current_state) == False).sum() == 0} with {current_state.sum()} in {step * 3} step")
coverage(current_state)

# Simulated annealing

In [ ]:
def probability(current_fitness, new_fitness, t):
    if new_fitness > current_fitness:
        return 1
    return math.exp(-(current_fitness - new_fitness) / t) / 2

In [ ]:
current_state = [False for _ in range(NUM_SETS)]
current_state = np.expand_dims(current_state, 0).transpose()
fitness_current = fitness(current_state)
print(f"Start fitness {fitness_current}")

max_step = 5_000
count_steady = max_step/10
count_equal = 0

In [ ]:
max_temp = max_step / 10
temperature1 = [1-(i+1)/max_step for i in range(max_step)]
temperature2 = [max_temp/(i+1) for i in range(max_step)]

temperature = temperature2

In [ ]:
for step in range(max_step):
    count_equal += 1
    state_new = tweak(current_state)
    fitness_new = fitness(state_new)
    prob = probability(fitness_current, fitness_new, temperature[step])
    if random() < prob:
        count_equal = 0
        current_state, fitness_current = state_new, fitness_new
        print(f"Fit {fitness_current} Step {step} Prob {prob}")
    if count_equal > count_steady:
        print("steady state reached")
        break

print(f"Resolved {(coverage(current_state) == False).sum() == 0} with {current_state.sum()} in {step} step")
coverage(current_state)